In [7]:
import paho.mqtt.client as mqtt
import paho.mqtt.publish as publish

from pymongo import MongoClient
from bson.objectid import ObjectId

import time,datetime
import os 
import schedule

import pandas as pd
import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import style

from copy import deepcopy
from IPython.display import clear_output

In [8]:
def realtimetotalstep():
    conn = MongoClient('120.126.136.17')
    db = conn.Tracker
    collection = [db.User_Info]# db.db2, db.dn2, db.james, db.leo
    userInfo = []
    for col in collection:
        cursor = col.find({})
        df_user = pd.DataFrame(list(cursor))
    collection = db.realtimetotalsteps
    collection.delete_many({})
    for username in df_user['username']:
        if username == 'pitest' or username == 'jack' or username == 'manager':
            continue
        collection = [db[username]]
        clean_data = []
        end = datetime.datetime.now().timestamp()
        start = (datetime.datetime.now().replace(hour=0,minute=0,second=0,microsecond=0)).timestamp()
        for col in collection:
            cursor = col.find({'timestamp': {'$gte': start, '$lt': end}})
            df = pd.DataFrame(list(cursor))
        df.replace('', np.nan, inplace=True)
        df.fillna(method='ffill', inplace=True)
        if 'hr_value' in df:
            df['hr_value'] = df['hr_value'].astype(float)
        if 'o2_value' in df:
            df['o2_value'] = df['o2_value'].astype(float)
        if 'latitude' in df:
            df['latitude'] = df['latitude'].astype(float)
        if 'longitude' in df:
            df['longitude'] = df['longitude'].astype(float)
        if 'step_value' in df:
            df['step_value'] = df['step_value'].astype(float)
        if 'hr_value' in df:    
            df = df[(df['hr_value'] != 0)]
        if 'step_value'  in df:
            totalstep = df['step_value'].sum()
            hourtotalstep = df.loc[df['hour'] == datetime.datetime.now().hour,['step_value']].values.sum()
        else:
            totalstep = 0
            hourtotalstep = 0
        now = datetime.datetime.now()
        dateStr = now.strftime('%Y-%m-%d-%H-%M')
        lis = dateStr.split('-')
        lis = list(map(int, lis))
        collection = db.realtimetotalsteps# db.db2, db.dn2, db.james, db.leo
        mydict = { 'year':lis[0], 'month':lis[1], 'day':lis[2], 'hour':lis[3], 'minute':lis[4],
                 'totalsteps':totalstep,'hourtotalstep':hourtotalstep,'user':username}
        
        collection.insert_one(mydict)
        conn.close()

In [9]:
def getRankData():

    t = time.localtime(time.time())
    year = t.tm_year
    month = t.tm_mon
    day = t.tm_mday
    hour = t.tm_hour
    rlist = []

    client=mqtt.Client()
    client.connect("120.126.136.17",1883,60)
    topic = ""
    mongo = MongoClient('120.126.136.17', 27017)
    db = mongo['Tracker']


    collection = db['User_Info']
    friendCol = collection.find()

    for x in friendCol:
        name = x['username']
        if name!= 'pitest' and name!= 'jack' and name!= 'manager':
            rlist.append(name)
    rsize=len(rlist)
    
    collection = db['rank']
    
    data = collection.find({'year':year,'month':month,'day':day,'hour':hour})

    #排名
    for x in data:
        topic = x['user']
        print(topic)
        rank =1
        pre=99999
        pro=-99999
        for name in rlist:
            if topic != name:
                if x[name]>0 and x[name]<pre:
                    rank=rank+1
                    pre=x[name]                 
                if x[name]<0 and x[name]>pro:
                    pro=x[name]
        if rank==1 and pre==99999:
            rank_mes_top = str(rank)+"/"+str(rsize)
            rank_mes_top = '%-7s' % rank_mes_top
            rank_mes_top = "排名 "+ rank_mes_top
            rank_mes_bot = "第"+str(rank)+"名,好棒棒"
            rank_mes=rank_mes_top+rank_mes_bot
            print(rank_mes)
            client.publish(topic,rank_mes) 
        elif pro==-99999:
            rank_mes_top = str(rank)+"/"+str(rsize)
            rank_mes_top = '%-7s' % rank_mes_top
            rank_mes_top = "排名 "+ rank_mes_top
            rank_mes_bot = "第"+str(rank)+"名,太混囉"
            rank_mes=rank_mes_top+rank_mes_bot
            print(rank_mes)
            client.publish(topic,rank_mes)
        else:
            rank_mes_top = str(rank)+"/"+str(rsize)
            rank_mes_top = '%-7s' % rank_mes_top
            rank_mes_top = "排名 "+ rank_mes_top
            rank_mes_bot = "第"+str(rank)+"名,再加油"
            rank_mes=rank_mes_top+rank_mes_bot
            print(rank_mes)
            client.publish(topic,rank_mes)
        time.sleep(0.01)
            
    time.sleep(5)
    print("======================")
    
    #前一名
    data = collection.find({'year':year,'month':month,'day':day,'hour':hour})
    
    for x in data:
        topic = x['user']
        print(topic)
        rank =1
        pre=99999
        pro=-99999
        for name in rlist:
            if topic != name:
                if x[name]>0 and x[name]<pre:
                    rank=rank+1
                    pre=x[name]                 
                if x[name]<0 and x[name]>pro:
                    pro=x[name]
        if rank==1 and pre==99999:
            rank_mes = "保持冠軍喔!"
            print(rank_mes)
            client.publish(topic,rank_mes) 
        else:
            rank_mes_top = "與前一名落差"
            rank_mes_bot = str(int(pre))+"步"
            rank_mes=rank_mes_top+rank_mes_bot
            print(rank_mes)
            client.publish(topic,rank_mes)
            
    time.sleep(5)
    print("======================")
    
    #後一名
    data = collection.find({'year':year,'month':month,'day':day,'hour':hour})
    
    for x in data:
        topic = x['user']
        print(topic)
        rank =1
        pre=99999
        pro=-99999
        for name in rlist:
            if topic != name:
                if x[name]>0 and x[name]<pre:
                    rank=rank+1
                    pre=x[name]                 
                if x[name]<0 and x[name]>pro:
                    pro=x[name]
        if pro==-99999:
            rank_mes = "吊車尾加把勁"
            print(rank_mes)
            client.publish(topic,rank_mes) 
        else:
            rank_mes_top = "與後一名落差"
            rank_mes_bot = str(abs(int(pro)))+"步"
            rank_mes=rank_mes_top+rank_mes_bot
            print(rank_mes)
            client.publish(topic,rank_mes)
            
    mongo.close()
    client.disconnect()

In [10]:
def getHourData():

    t = time.localtime(time.time())
    year = t.tm_year
    month = t.tm_mon
    day = t.tm_mday
    hour = t.tm_hour

    client=mqtt.Client()
    client.connect("120.126.136.17",1883,60)
    topic = ""
    

    mongo = MongoClient('120.126.136.17', 27017)
    db = mongo['Tracker']
    collection = db['prediction']
    
    data = collection.find({'year':year,'month':month,'day':day,'hour':hour,'type':'hourly'})
    
    for x in data:
        topic = x['user']
        print(topic)
        top = '步數:%5s步' % int(x['totalsteps'])
        bot = '目標:%5s步' % x['prediction']
        mes = top+bot
        print(mes)
        client.publish(topic,mes)
        time.sleep(0.01)
    
    mongo.close()
    client.disconnect()

In [11]:
def lazyreminder():
    conn = MongoClient('120.126.136.17')
    db = conn.Tracker 
    server = "120.126.136.17"
    port = 1883 #port
    
    collection = [db.prediction]
    userInfo = []
    for col in collection:
        cursor = col.find({})
        dfprediction = pd.DataFrame(list(cursor))
    dfprediction = dfprediction.loc[(dfprediction['year'] == datetime.datetime.now().year) & 
                                    (dfprediction['month'] == datetime.datetime.now().month) &
                                    (dfprediction['hour'] == datetime.datetime.now().hour + 1) &
                                    (dfprediction['day'] == datetime.datetime.now().day)]

    dfrealtimetotalsteps = pd.DataFrame({})
#     print(dfprediction)
    print(dfprediction)
    while dfprediction.shape[0] != dfrealtimetotalsteps.shape[0]:
        print(dfprediction.shape, dfrealtimetotalsteps.shape)
        time.sleep(3)
        collection = [db.realtimetotalsteps]# db.db2, db.dn2, db.james, db.leo
        userInfo = []
        for col in collection:
            cursor = col.find({})
            dfrealtimetotalsteps = pd.DataFrame(list(cursor))
#     print(dfprediction['prediction'])
    result = (dfprediction['prediction'] * datetime.datetime.now().minute / 60).values < \
                    (dfrealtimetotalsteps['hourtotalstep'].values * datetime.datetime.now().minute / 60)
    diff = (dfprediction['prediction'] * datetime.datetime.now().minute / 60).values \
                - (dfrealtimetotalsteps['hourtotalstep'].values * datetime.datetime.now().minute / 60)
    conn.close()
    lis = list(zip(dfprediction['user'].values,result,diff))
    for name,result,diff in lis:
        if result == False:
            mqttc = mqtt.Client()
            mqttc.connect(server, port)
            if math.isnan(diff):
                diff = 0
            mqttc.publish('standup' + name, "你在偷懶歐! 還差" + str(int(diff)) +"步")

In [17]:
lazyreminder()

Empty DataFrame
Columns: [_id, day, hour, month, prediction, totalsteps, type, user, year]
Index: []


KeyError: 'hourtotalstep'

In [16]:
# getHourData()
getRankData()
# realtimetotalstep()

t
排名 3/5    第3名,太混囉
jeng
排名 3/5    第3名,太混囉
leo
排名 3/5    第3名,太混囉
james
排名 1/5    第1名,好棒棒
shuan
排名 2/5    第2名,再加油
t
與前一名落差259步
jeng
與前一名落差259步
leo
與前一名落差259步
james
保持冠軍喔!
shuan
與前一名落差6742步
t
吊車尾加把勁
jeng
吊車尾加把勁
leo
吊車尾加把勁
james
與後一名落差6742步
shuan
與後一名落差259步


In [13]:
schedule.every(1).minutes.do(realtimetotalstep)
schedule.every(3).minutes.do(lazyreminder)
# schedule.every(4).minutes.do(getHourData)

Every 3 minutes do lazyreminder() (last run: [never], next run: 2019-05-13 14:07:59)

In [14]:
while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 

In [ ]:
conn = MongoClient('120.126.136.17')
db = conn.Tracker
collection = [db.prediction]# db.db2, db.dn2, db.james, db.leo
user = []

for col in collection:
    cursor = col.find({})
    prediction = pd.DataFrame(list(cursor))
timedf = prediction.loc[(prediction['month'] == datetime.datetime.now().month) 
                        & (prediction['day']== datetime.datetime.now().day) 
                        & (prediction['hour'] == datetime.datetime.now().hour)].reset_index()
timedf.drop(['index','_id'], inplace=True, axis=1)
timedf = timedf.set_index('user')
# remove duplicate index
timedf = timedf.loc[~timedf.index.duplicated(keep='last')]
for i in timedf.index:
    timedf[i] = 0
for index, row in timedf.iterrows():
    for i in timedf.index:
        timedf.loc[index,i] = timedf.loc[i,'totalsteps'] - timedf.loc[index,'totalsteps']
timedf.drop(['prediction', 'totalsteps', 'type'], inplace=True, axis=1)
timedf.reset_index(inplace=True)
records = timedf.to_dict('records')
if records != []:
    collection = db.rank
    collection.insert_many(records)
else:
    print('records', records)
conn.close()